In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.optim as optim
torch.manual_seed(1)

import random
random.seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.cuda.manual_seed_all(1)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv


In [2]:
X = pd.read_csv('/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv')
y = X['Category']
X = X.drop(['Category'], axis = 1)
X = X.to_numpy().reshape(-1, 1, 28, 28)

In [3]:
X = torch.FloatTensor(X).to(device)
y = torch.LongTensor(y.to_numpy()).to(device)

In [4]:
# parameters
training_epochs = 10
batch_size = 100

In [5]:
from torch.utils.data import  TensorDataset
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=TensorDataset(X, y),
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
resnet = models.resnet18(pretrained=True)

# change the number of classes 
resnet.fc.out_features = 10
    
# for param in resnet.parameters(): 
#     param.requires_grad = False
resnet.layer1.requires_grad=False
resnet.layer2.requires_grad=False
resnet.layer3.requires_grad=False
resnet.layer4.requires_grad=False
    
def resnetplus(model):
    linear1 = torch.nn.Linear(256, 128)
    linear2 = torch.nn.Linear(128, 10)

    torch.nn.init.xavier_normal_(linear1.weight)
    torch.nn.init.xavier_normal_(linear2.weight)

    relu = torch.nn.ReLU()
    dropout = torch.nn.Dropout(0.3)
    res_model = torch.nn.Sequential(model, relu, dropout,
                                    linear1,relu, dropout,
                                   linear2).to(device)
    return res_model
    
resnetplus(resnet)
# .99470

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [7]:
# resnet = models.resnet18(pretrained=True)


# # change the number of classes 
# # resnet.fc.out_features = 256
    
# # for param in resnet.parameters(): 
# #     param.requires_grad = False

# resnet.layer1.requires_grad=False
# resnet.layer2.requires_grad=False
# resnet.layer3.requires_grad=False
# resnet.layer4.requires_grad=False
    
# def resnetplus(model):
#     linear1 = torch.nn.Linear(1000, 256)
#     linear2 = torch.nn.Linear(256, 128)
#     linear3 = torch.nn.Linear(128, 10)

#     torch.nn.init.xavier_normal_(linear1.weight)
#     torch.nn.init.xavier_normal_(linear2.weight)
#     torch.nn.init.xavier_normal_(linear3.weight)

#     relu = torch.nn.ReLU()
#     dropout = torch.nn.Dropout(0.3)
#     res_model = torch.nn.Sequential(model, 
#                                     linear1, relu, dropout,
#                                    linear2, relu, dropout,
#                                    linear3 ).to(device)
#     return res_model
    
# resnetplus(resnet)
# # .99550

In [8]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-4)

In [10]:
resnet.train()
total_batch = len(data_loader)      

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = transforms.Resize(224)(X)
        X = X.repeat(1, 3, 1, 1)
        X = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(X)
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = resnet(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.117140181
[Epoch:    2] cost = 0.0185586158
[Epoch:    3] cost = 0.0117252143
[Epoch:    4] cost = 0.00987461396
[Epoch:    5] cost = 0.00851773191
[Epoch:    6] cost = 0.00543667004
[Epoch:    7] cost = 0.00786158163
[Epoch:    8] cost = 0.00818952173
[Epoch:    9] cost = 0.00509762717
[Epoch:   10] cost = 0.00297881337


In [11]:
sample =pd.read_csv('/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv')

In [12]:
test=pd.read_csv('/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv')
test = test.to_numpy().reshape(-1, 1, 28, 28)
test = torch.FloatTensor(test).to(device)

In [13]:
resnet.eval()
i = 0
with torch.no_grad():
    for t in test:
        t = transforms.Resize(224)(t)
        t = t.repeat(1, 3, 1, 1)
        t = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(t)
        p = torch.argmax(resnet(t), 1).cpu()
        p = p.detach().numpy()
        sample['Category'][i] = p
        i = i + 1   


In [14]:
sample.head()

,Id,Category
0,0,6
1,1,1
2,2,0
3,3,0
4,4,9


In [15]:
sample.to_csv('submit.csv', index=False)